In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo

$$
\begin{split}
J_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu \nu | \kappa \lambda]\\
J_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \phi_i | \kappa \lambda]
\end{split}
$$

In [2]:
def build_J(Fouridx,C,N):
    J = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        J[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,nu,kappa,lamda]
    return 2*J # needs factor 2 because its spin restricted

$$
\begin{split}
K_{\kappa \lambda} & = \sum_{i=1}^N  \sum_{\mu}^{M} \sum_{\nu}^{M} c_{i, \mu} c_{i, \nu} [\mu  \kappa| \nu  \lambda]\\
K_{\kappa \lambda} & = \sum_{i=1}^N  [\phi_i \kappa | \phi_i \lambda]
\end{split}
$$

In [3]:
def build_K(Fouridx,C,N):
    K = np.zeros(C.shape)
    for kappa in range(0,Fouridx.shape[0]):
        for lamda in range(0,Fouridx.shape[0]):
            for i in range(0,N): # all occupied
                for mu in range(0,Fouridx.shape[0]): # all basis functions
                    for nu in range(0,Fouridx.shape[0]): # all basis functions
                        K[kappa][lamda] += C[mu,i]*C[nu,i]*Fouridx[mu,kappa,nu,lamda]
    return 2*K # needs factor 2 because its spin restricted

In [4]:
mol = gto.Mole()
mol.atom = """
    Ne    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #mol.basis = "ccpvdz", mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 in the case of  case 16 distinct elements
eri = mol.intor('int2e')

In [5]:
mf = scf.RHF(mol)
mf.kernel()
J = mf.get_j()
K = mf.get_k()
C = mf.mo_coeff
N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)

converged SCF energy = -128.473876870668


In [6]:
print(J)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   2.27809637e-17  6.47240754e-18  4.93165923e+00  1.54705433e-17
   2.34586321e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.27809637e-17
   1.28730330e+01  1.11547358e-17  1.54705433e-17  4.93165923e+00
  -3.33257869e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  6.47240754e-18
   1.11547358e-17  1.28730330e+01  2.34586321e-18 -3.33257869e-18
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
   1.54705433e-17  2.3458632

In [7]:
J_2 = build_J(eri,C,N)
print(J_2)

[[ 2.34250003e+01  4.30815331e+00  3.26823406e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 4.30815331e+00  1.24357919e+01  8.60782026e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 3.26823406e+00  8.60782026e+00  8.83395486e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.28730330e+01
   2.34187669e-17  5.82758668e-18  4.93165923e+00  1.50704102e-17
   2.16840434e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.34187669e-17
   1.28730330e+01  3.68628739e-18  1.50704102e-17  4.93165923e+00
  -2.71050543e-18]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.82758668e-18
   3.68628739e-18  1.28730330e+01  2.16840434e-18 -2.71050543e-18
   4.93165923e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.93165923e+00
   1.63714528e-17  2.2361669

In [8]:
print(J[0,0]/J_2[0,0], J[1,1]/J_2[1,1])

0.9999999999999996 0.9999999999999997


In [9]:
print(K)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
   1.64562251e-16  5.87161233e-17  1.68027926e+00  1.32092879e-16
   2.55055532e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.64562251e-16
   3.08225045e+00  1.44811936e-16  3.06797825e-16  1.68027926e+00
  -4.60164861e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.87161233e-17
   1.44811936e-16  3.08225045e+00  2.55178699e-17 -1.29330499e-16
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   3.06797825e-16  2.5517869

In [10]:
K_2 = build_K(eri,C,N)
print(K_2)

[[ 1.24802204e+01  2.01345026e+00  1.64951921e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 2.01345026e+00  3.65352126e+00  2.98783781e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 1.64951921e+00  2.98783781e+00  2.64819848e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.08225045e+00
   2.07407876e-16  6.03155221e-17  1.68027926e+00  1.63389267e-16
   2.72676846e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.15973073e-16
   3.08225045e+00  1.70761842e-16  3.15719673e-16  1.68027926e+00
  -7.07984019e-17]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.83300769e-17
   9.71445147e-17  3.08225045e+00  2.73218947e-17 -1.14491749e-16
   1.68027926e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.68027926e+00
   3.14527050e-16  2.7348999